In [ ]:
src_path = "../src/"
lib_path = "../lib/"
input_path = "../../kaggle/input/home-credit-default-risk/"

In [3]:
%conda install apple::tensorflow-deps --force-reinstall

Channels:
 - conda-forge
 - apple
Platform: osx-arm64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 24.11.3
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/homebrew/Caskroom/miniforge/base/envs/kerasenv

  added / updated specs:
    - apple::tensorflow-deps





Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install tensorflow-macos==2.12.0

  Using cached tensorflow_macos-2.12.0-cp310-cp310-macosx_12_0_arm64.whl.metadata (3.2 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached numpy-1.23.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.3 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached termcolor-3.0.1-py3-none-any.whl.metadata (6.1 kB)
  Using cached wrapt-1.14.1-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow_estimator-2.12.0-py2.py3-none-any.whl.metadata (1.3 kB)
  Using cached keras-2.12.0-py2.py3-none-any.whl.metadata (1.4 kB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/opt/homebrew/Caskroom/miniforge/base/envs/kerasenv/lib/python3.10/site-packages/flatbuffers-25.2.10.dist-info/METADATA'

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install numpy==1.23.5

  Using cached numpy-1.23.5-cp310-cp310-macosx_11_0_arm64.whl.metadata (2.3 kB)
Using cached numpy-1.23.5-cp310-cp310-macosx_11_0_arm64.whl (13.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dm-tree 0.1.9 requires wrapt>=1.11.2, which is not installed.
jax 0.4.30 requires opt-einsum, which is not installed.
line 0.8.2 requires rsa, which is not installed.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tensorflow-metal==0.8.0

  Using cached tensorflow_metal-0.8.0-cp310-cp310-macosx_12_0_arm64.whl.metadata (776 bytes)
Using cached tensorflow_metal-0.8.0-cp310-cp310-macosx_12_0_arm64.whl (1.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall tensorflow-macos -y

In [4]:
pip list

Package                   Version
------------------------- --------------
absl-py                   2.2.2
aggdraw                   1.3.19
anyio                     4.9.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
backcall                  0.2.0
beautifulsoup4            4.13.4
bleach                    6.2.0
Brotli                    1.1.0
cached-property           1.5.2
category_encoders         2.6.4
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.2
curve                     0.1.0
cycler                    0.12.1
debugpy                   1.8.14
decorator                 5.1.1
defusedxml                0.7.1
dm-tree         

In [ ]:
import sys

import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Input
from keras.metrics import AUC
from keras.models import Sequential
from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

pd.set_option("display.max_columns", 200)
tf.random.set_seed(42)
set_config(transform_output="pandas")

sys.path.insert(0, src_path)
data_processing = joblib.load(lib_path + "data_processing.joblib")
preprocessor = data_processing["preprocessor"]
non_co_cols = data_processing["non_co_cols"]
merged_application_test = data_processing["merged_application_train"]
del sys.path[0]

In [ ]:
def plot_history(history: keras.callbacks.History) -> None:
    """_summary_.

    _extended_summary_

    Args:
        history (_type_): _description_

    """
    # Get all metrics (excluding validation metrics)
    metrics = [m for m in history.history.keys() if not m.startswith("val_")]

    # Create a figure for each metric with training and validation on the same plot
    for metric in metrics:
        val_metric = f"val_{metric}"
        has_validation = val_metric in history.history.keys()

        # Create figure
        plt.figure(figsize=(10, 6))

        # Plot training metric in blue
        epochs = range(1, len(history.history[metric]) + 1)  # Start epochs at 1
        plt.plot(epochs, history.history[metric], "b-", label=f"Training {metric}")

        # Plot validation metric in red if available
        if has_validation:
            plt.plot(
                epochs, history.history[val_metric], "r-", label=f"Validation {metric}"
            )

        # Set title and labels
        plt.title(f"{metric.capitalize()} over epochs")
        plt.xlabel("Epoch")
        plt.ylabel(metric.capitalize())

        # Set intelligent integer ticks on x-axis based on number of epochs
        num_epochs = len(epochs)
        if num_epochs <= 20:
            # For few epochs, show all ticks
            plt.xticks(epochs)
        else:
            # For many epochs, show at most ~10 ticks evenly spaced
            step = max(1, num_epochs // 10)
            tick_positions = list(range(1, num_epochs + 1, step))
            # Always include the first and last epoch
            if tick_positions[-1] != num_epochs:
                tick_positions.append(num_epochs)
            plt.xticks(tick_positions)

        # Add grid and legend
        plt.grid(True)
        plt.legend()

        # Show the plot
        plt.tight_layout()
        plt.show()

In [ ]:
print(f"TensorFlow {tf.__version__}")
print(f"Keras {keras.__version__}")

gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")
print("Number of GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print(tf.reduce_sum(tf.random.normal([1000, 1000])))
tf.config.list_physical_devices('GPU')

In [ ]:
# separate data
X = merged_application_test.drop(columns=["TARGET", "SK_ID_CURR"])
y = merged_application_test["TARGET"].copy()
# split data 80/10/10 for training, validation, and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, train_size=0.5, test_size=0.5, random_state=42)
# transform data
X_train = preprocessor.fit_transform(X_train)
X_val = preprocessor.transform(X_val)
X_test = preprocessor.transform(X_test)
# drop columns with collinear relationships (Pearson's correlation coefficients > 0.8)
X_train = X_train.drop(columns=non_co_cols)
X_val = X_val.drop(columns=non_co_cols)
X_test = X_test.drop(columns=non_co_cols)
# scale remaining data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight="balanced", classes=np.unique(y_train), y=y_train,
)
class_weights = class_weights.round(1)
class_weight_dict = dict(enumerate(class_weights))
class_weight_dict

# Model binary_crossentropy

In [ ]:
from keras.metrics import Recall
from keras.optimizers import Adam


def create_model_bce():
    model = Sequential()

    model.add(Input(shape=(X_train.shape[1],)))  # Input layer

    model.add(Dense(128, activation="relu"))  # Hidden layer 1
    model.add(Dropout(0.2))

    model.add(Dense(64, activation="relu"))  # Hidden layer 1
    model.add(Dropout(0.2))

    model.add(Dense(32, activation="relu"))  # Hidden layer 2
    model.add(Dropout(0.2))

    model.add(Dense(16, activation="relu"))  # Hidden layer 3
    model.add(Dropout(0.2))

    model.add(Dense(8, activation="relu"))  # Hidden layer 4
    model.add(Dropout(0.2))

    model.add(Dense(1, activation="sigmoid"))  # Output layer

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss="bce",
        metrics=["accuracy", Recall(name="recall"), AUC(name="auc")],
    )
    return model


model_bce = create_model_bce()

history_bce = model_bce.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    class_weight=class_weight_dict,
    epochs=100,
    batch_size=2048,
    callbacks=[
        EarlyStopping(
            patience=10,
            restore_best_weights=True,
            monitor="val_auc",
            mode="max",
        ),
    ],
)

In [ ]:
plot_history(history=history_bce)

In [ ]:
import keras_tuner as kt


def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1],)))  # Input layer

    # Tune the number of layers and units in each layer
    for i in range(hp.Int('num_layers', min_value=1, max_value=5)):
        model.add(Dense(units=hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                        activation='relu'))
        model.add(Dropout(hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='sigmoid'))  # Output layer

    # Tune the learning rate
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(
        optimizer=Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy', Recall(name='recall'), AUC(name='auc')]
    )
    return model

In [ ]:
model_tune = build_model()

history_tune = model_tune.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    class_weight=class_weight_dict,
    epochs=100,
    batch_size=2048,
    callbacks=[
        EarlyStopping(
            patience=10,
            restore_best_weights=True,
            monitor="val_auc",
            mode="max",
        ),
    ],
)

In [ ]:
# model.save("mnist_model1.keras")

In [ ]:
# from keras.models import load_model

# loaded_model = load_model("mnist_model1.keras")

preds = model_bce.predict(X_test)

# Apply a threshold to convert probabilities to classifications
threshold = 0.5  # You can adjust this threshold as needed
y_pred = (preds > threshold).astype(int)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_test, y_pred);